## Installation 

If you have not installed snowflake-snowpark-python you can install it here. Make sure you are in the right conda environment.

If you are not sure you can try:

```bash
! which python
```

In [119]:
! which python

/opt/anaconda3/envs/dev2/bin/python


Install the library

In [117]:
# !conda install snowflake-snowpark-python -y

# THE **E**
## Snowpark Usage - Example

In [87]:
from snowflake.snowpark import Session

Example of how the `snow.cfg` file looks like

```yaml
[SNOW]
ACCOUNT=WPA36811
USER=tarek
PASSWORD=yourpassowrd
WAREHOUSE=COMPUTE_WH
DATABASE=TECHCATALYST_DE
SCHEMA=PUBLIC
ROLE=DE
```

In [118]:
import configparser

config = configparser.ConfigParser()
config.read('snow.cfg')
config.sections()

['SNOW']

Create a dictionary (key-value) of the parameters

In [90]:
params1 = dict(config['SNOW'])

Passing the parameters as a dictionary to Session

In [96]:
session1 = Session.builder.configs(params1).create()

In [123]:
print(session1.get_current_user())
print(session1.get_current_database())
print(session1.get_current_schema())

"tarek"
"TECHCATALYST_DE"
"PUBLIC"


Example using the `table` methods to call a specific table. In this example I am calling the `INS_ACCIDENTS` table

In [124]:
accidents = session1.table("INS_ACCIDENTS")
accidents.show(5)

-------------------------------------------------------------------------------------------------------------------------------------------------------
|"ACCIDENT_ID"  |"POLICYHOLDER_ID"  |"VEHICLE_ID"  |"ACCIDENT_TYPE"  |"ACCIDENT_DATE"  |"ESTIMATED_COST"  |"ACTUAL_REPAIR_COST"  |"AT_FAULT"  |"DUI"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------
|1              |4333               |176           |3                |2022-03-30       |6694              |7480                  |True        |True   |
|2              |4547               |6391          |4                |2023-09-27       |2190              |3355                  |True        |False  |
|3              |6686               |6974          |7                |2022-06-13       |4995              |7123                  |False       |False  |
|4              |1300               |4037          |7                |2020-01-21       |

Note, `accidents` is a Snowpark object (table). There is a difference between Snowpark DataFrames and Panda DataFrames. For now, you will convert to a Pandas DataFrame to do your transformations. Once done, you will convert back to a Snowpark DataFrame and write to the Database.

In [125]:
type(accidents)

snowflake.snowpark.table.Table

In [126]:
accidents_df = accidents.to_pandas()

In [127]:
accidents_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ACCIDENT_ID         10000 non-null  int16 
 1   POLICYHOLDER_ID     10000 non-null  int16 
 2   VEHICLE_ID          10000 non-null  int16 
 3   ACCIDENT_TYPE       10000 non-null  int8  
 4   ACCIDENT_DATE       10000 non-null  object
 5   ESTIMATED_COST      10000 non-null  int16 
 6   ACTUAL_REPAIR_COST  10000 non-null  int16 
 7   AT_FAULT            10000 non-null  bool  
 8   DUI                 10000 non-null  bool  
dtypes: bool(2), int16(5), int8(1), object(1)
memory usage: 205.2+ KB


# The **T**
## Your Work Here (Transformations)

# THE **L**
## Write to Snowflak

To avoid mistakes, make sure you use your schema. 

In [128]:
yourschema = 'tatwan'
location = f'techcatalyst_de.{yourschema}'
print(location)

techcatalyst_de.tatwan


Convert the Pandas DataFrame into a Snowpark DataFrame

In [131]:
accidents_sdf = session1.create_dataframe(accidents_df)

In [133]:
print(type(accidents_df))
print(type(accidents_sdf))

<class 'pandas.core.frame.DataFrame'>
<class 'snowflake.snowpark.table.Table'>


When using the `write.mode()` method there are different options:

* ”`append`”: Append data of this DataFrame to the existing table. Creates a table if it does not exist.

* ”`overwrite`”: Overwrite the existing table by dropping old table.

* ”`truncate`”: Overwrite the existing table by truncating old table.

* ”`errorifexists`”: Throw an exception if the table already exists.

* ”`ignore`”: Ignore this operation if the table already exists.

* Default value is “`errorifexists`”.

In [135]:
# example using overwrite

accidents_sdf.write.mode("overwrite").save_as_table(f"{location}.tatwan_test")